In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pyqg
from pyqg_generative.models.ols_model import OLSModel
from pyqg_generative.models.mean_var_model import MeanVarModel
from pyqg_generative.tools.stochastic_pyqg import stochastic_QGModel
from time import time
%load_ext autoreload
%autoreload 3

/ext3/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# OLS Model

In [2]:
model = OLSModel()

In [3]:
ds_test = xr.open_mfdataset('/scratch/pp2681/pyqg_generative/Reference-Default-scaled/eddy/Operator1-32/[0-1].nc', combine='nested', concat_dim='run')
ds_train = xr.open_mfdataset('/scratch/pp2681/pyqg_generative/Reference-Default-scaled/eddy/Operator1-32/[1-2].nc', combine='nested', concat_dim='run')

In [4]:
model.fit(ds_train, ds_test, num_epochs=1)
model.fit(ds_train, ds_test, num_epochs=1)

Training starts on device cpu, number of samples 172
[1/1] [1.27/0.00] Loss: [1.321, 1.018]
Training starts on device cpu, number of samples 172
[1/1] [1.03/0.00] Loss: [1.318, 1.019]


In [5]:
model.predict(ds_test)

<xarray.Dataset>
Dimensions:                   (run: 2, time: 86, lev: 2, y: 32, x: 32)
Dimensions without coordinates: run, time, lev, y, x
Data variables:
    q_forcing_advection       (run, time, lev, y, x) float32 3.263e-13 ... -1...
    q_forcing_advection_mean  (run, time, lev, y, x) float32 3.263e-13 ... -1...
    q_forcing_advection_var   (run, time, lev, y, x) float32 0.0 0.0 ... -0.0

In [6]:
m = stochastic_QGModel({'parameterization': model, 'tmax':10*86400}, sampling_type='constant', nsteps=2)
%time m.run()

CPU times: user 2.9 s, sys: 3.01 s, total: 5.91 s
Wall time: 495 ms


# MeanVar model

In [12]:
model = MeanVarModel()

In [13]:
model.fit(ds_train, ds_test, num_epochs=1)
model.fit(ds_train, ds_test, num_epochs=1)

Training starts on device cpu, number of samples 172
[1/1] [1.07/0.00] Loss: [1.313, 1.017]
Training starts on device cpu, number of samples 172
[1/1] [1.01/0.00] Loss: [4.722, 4.912]
Training starts on device cpu, number of samples 172
[1/1] [1.04/0.00] Loss: [1.307, 1.017]
Training starts on device cpu, number of samples 172
[1/1] [0.96/0.00] Loss: [4.715, 4.924]


In [14]:
model.predict(ds_test)

<xarray.Dataset>
Dimensions:                   (run: 2, time: 86, lev: 2, y: 32, x: 32)
Dimensions without coordinates: run, time, lev, y, x
Data variables:
    q_forcing_advection       (run, time, lev, y, x) float64 -5.108e-12 ... 2...
    q_forcing_advection_mean  (run, time, lev, y, x) float32 2.387e-13 ... -1...
    q_forcing_advection_var   (run, time, lev, y, x) float32 5.498e-23 ... 4....

In [16]:
offline = model.test_offline(ds_test)

In [31]:
m = stochastic_QGModel({'parameterization': model, 'tmax':10*86400}, sampling_type='constant', nsteps=2)
%time m.run()

CPU times: user 5.55 s, sys: 5.45 s, total: 11 s
Wall time: 921 ms


# Typical runtime
- 16 mins for 20 years run for OLS Model on 1 core. 
- 32 min for GZ model. 
- 16 mins for cGAN without separation

In [98]:
m = stochastic_QGModel({'parameterization': model, 'dt': 14400, 'tmax': 360*86400})
t = time()
m.run()
print('Time for 20 years run in mins=', (time()-t)*20/60)

Time for 20 years run in mins= 11.250957330067953
